In [9]:
import tweepy
import csv
import pandas as pd
import time
import sys

### Loading

In [10]:
dates = ['2020-03-27', '2020-03-28', '2020-03-29', '2020-03-30', '2020-03-31',
        '2020-04-01', '2020-04-02']

In [16]:
dict_df = dict()
for i in range(len(dates) - 1):
    csv_name = 'data_10000/' + 'data_'+ str(dates[i]) + '_' + str(dates[i+1]) + '.csv'
    df_name = 'df_' + str(dates[i])
    dict_df[df_name] = pd.read_csv(csv_name)

,timestamp,username,text,is_retweeted,user_retweeted
0,2020-03-27 23:59:47,Javou6,RT @LachaudB: Ils veulent utiliser le #coronav...,True,LachaudB
1,2020-03-27 23:59:23,JudithLearn,RT @afpfr: Le #confinement a eu pour conséquen...,True,afpfr
2,2020-03-27 23:59:20,Colemos59,RT @AllanBARTE: Qui aurait pu prévoir ça ? Qui...,True,AllanBARTE
3,2020-03-27 23:59:14,BeHumanJ,RT @AllanBARTE: Qui aurait pu prévoir ça ? Qui...,True,AllanBARTE
4,2020-03-27 23:59:12,claireparisjazz,RT @30millionsdamis: 😱😡#OnMarcheSurLaTête‼️A p...,True,30millionsdamis
...,...,...,...,...,...
9995,2020-03-27 09:58:37,kalliste06,RT @cathpellissier: Jour11 #confinement\nBleu\...,True,cathpellissier
9996,2020-03-27 09:58:32,MANGADO1,RT @Charlie_Hebdo_: 11ème jour de #confinement...,True,Charlie_Hebdo_
9997,2020-03-27 09:58:27,apirlot,RT @sandrafellous_: 🔴 Dans le 93 habite de la ...,True,sandrafellous_
9998,2020-03-27 09:58:24,Covid19Paris,RT @Charlie_Hebdo_: 11ème jour de #confinement...,True,Charlie_Hebdo_


In [ ]:
status=api.show_friendship(source_screen_name='ErmannoKilgore', target_screen_name='SamsonHuang4')
print(status[0].following, status[1].following)